# Entrepreneurial Pitch MDA and peruasiveness scoring

In this notebook metadiscursive score for persuasiveness is computed for the 2023 Rice business pitches.  The transcripts were obtained from [public YouTube videos](https://youtu.be/vqUVxbD2NLA?si=K-9FutgcCDdGKh89).

Persuasiveness score is defined as: percentage(interactive) - (pc_hedges + pc_emphatics + pc_att_markers).

In [ ]:
import numpy as np
import pandas as pd
#import plotly.express as px
import string
#import spacy
#nlp = spacy.load("en_core_web_sm")

from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import stem_text

#from gensim.summarization.summarizer import summarize
#from gensim.summarization import keywords

#import networkx as nx

import os
import tqdm

import re

import plotly.express as px

In [ ]:
class MetaDiscourseAnalysis:

  def __init__(self):
    self.CodeGloss = ["matter", "fact", "called", "defined", "example", "instance", "mean", "ie", "i.e.", "i.e", "know", "know", "namely", "another", "way", "put", "say", "specifically", "such", "as", "that", "mean", "means", "viz", "viz."];
    self.Endophorics = ["chapter", "part", "section", "example", "fig", "figure", "fig.", "page", "table", "above", "before", "below", "earlier", "later"];
    self.Evidentials = ["cite", "cited", "quoted", "quote", "according"];
    self.FrameMarkersSequencing = ["chapter", "section", "part", "finally", "final", "first", "second", "third", "firstly", "secondly", "thirdly", "fourth", "fifth", "subsequent", "subsequently", "then"];
    self.FrameMarkersLabels = ["all", "point", "stage", "far", "moment", "brief", "conclusion", "short", "summary", "now", "whole", "overall", "thus", "conclude", "repeat", "sum", "summarize"];
    self.FrameMarkersAnnounceGoals = ["this", "aim", "desire", "focus", "goal", "intend", "intention", "objective", "purpose", "seek", "want", "wish", "would", "like"];
    self.FrameMarkersShiftTopics = ["back", "digress", "regard", "move", "resume", "return", "revisit", "shift", "look", "closely", "turn", "regard"];
    self.FrameMarkers = self.FrameMarkersSequencing + self.FrameMarkersLabels + self.FrameMarkersAnnounceGoals + self.FrameMarkersShiftTopics
    self.Transition = ["accordingly", "according", "additionally", "again", "also", "alternatively", "although", "consequence", "result", "same", "time", "because", "besides", "but", "contrast", "by", "token", "consequently", "conversely", "equally", "equal", "even", "though", "further", "furthermore", "hence", "however", "addition", "contrast", "way", "leads", "likewise", "moreover", "nevertheless", "nonetheless", "contrary", "hand", "rather", "result", "similarly", "similar", "since", "so", "still", "result", "thereby", "therefore", "though", "thus", "whereas", "while", "yet"];
    self.AttitudeMarkers = ["admittedly", "agree", "agrees", "agreed", "amazed", "amazing", "amazingly", "appropriate", "appropriately", "astonished", "astonishing", "astonishingly", "correctly", "curious", "curiously", "desirable", "desirably", "disappointed", "disappointing", "disappointingly", "disagree", "disagreed", "disagrees", "dramatic", "dramatically", "essential", "essentially", "even", "expected", "expectedly", "fortunate", "fortunately", "hopeful", "hopefully", "important", "importantly", "inappropriate", "inappropriately", "interesting", "interestingly", "prefer", "preferable", "preferably", "preferred", "remarkable", "remarkably", "shocked", "shocking", "shockingly", "striking", "strikingly", "surprised", "surprising", "surprisingly", "unbelievable", "unbelievably", "understandable", "understandably", "unexpected", "unexpectedly", "unfortunate", "unfortunately", "unusual", "unusually", "usual"];
    self.Emphatics=["actually", "always", "believe", "believed", "believes", "beyond doub t", "certain", "certainly", "clear", "clearly", "conclusively", "decidedly", "definite", "definitely", "demonstrate", "demonstrated", "demonstrates", "doubtless", "establish", "established", "evident", "evidently", "find", "finds", "found", "fact", "incontestable", "incontestably", "incontrovertible", "incontrovertibly", "indeed", "indisputable", "indisputably", "know", "known", "must", "never", "obvious", "obviously", "course", "prove", "proved", "proves", "realize", "realized", "realizes", "really", "show", "showed", "shown", "shows", "sure", "surely", "think", "thinks", "thought", "truly", "true", "undeniable", "undeniably", "undisputedly", "undoubtedly", "doubt"];
    self.PersonMarkers = ["i", "we", "me", "my", "our", "mine", "us", "you", "your", "author", "authors", "author's", "writer", "writer's", "writers", "they", "them", "he", "she"];
    self.Engagement = ["reader", "readers", "add", "allow", "analyse", "apply", "arrange", "assess", "assume", "by the way", "calculate", "choose", "classify", "compare", "connect", "consider", "consult", "contrast", "define", "demonstrate", "determine", "do not", "develop", "employ", "ensure", "estimate", "evaluate", "find", "follow", "go", "have", "imagine", "incidentally", "increase", "input", "insert", "integrate", "key", "let", "let's", "lets", "look", "mark", "measure", "mount", "must", "need", "note", "notice", "observe", "one's", "ones", "order", "ought", "our ", "pay", "picture", "prepare", "recall", "recover", "refer", "regard", "remember", "remove", "review", "see", "select", "set", "should", "show", "suppose", "state", "take", "think", "turn", "us", "use", "we", "you", "your"];
    self.Hedges = ["about", "almost", "apparent", "apparently", "appear", "appeared", "appears", "approximately", "argue", "argued", "argues", "around", "assume", "assumed", "broadly", "certain", "extent", "level", "claim", "claimed", "claims", "could", "couldn't", "doubt", "doubtful", "essentially", "estimate", "estimated", "fairly", "feel", "feels", "felt", "frequently", "perspective", "generally", "guess", "indicate", "indicated", "indicates", "general", "most", "opinion", "view", "largely", "likely", "mainly", "may", "maybe", "might", "mostly", "often", "ought", "perhaps", "plausible", "plausibly", "possible", "possibly", "postulate", "postulated", "postulates", "presumable", "presumably", "probable", "probably", "quite", "rather ", "relatively", "roughly", "seems", "should", "sometimes", "somewhat", "suggest", "suggested", "suggests", "suppose", "supposed", "supposes", "suspect", "suspects", "tend t o", "tended to", "tends to", "knowledge", "typical", "typically", "uncertain", "uncertainly", "unclear", "unclearly", "unlikely", "usually", "would", "wouldn't", "wouldnt"];

  def authorial_stance(self, text):
    epsilon = 0.000000000001;
    words = strip_multiple_whitespaces(strip_punctuation(text.lower())).split()
    pc_person_markers = np.round(float(len([i for i in words if i in self.PersonMarkers])/(len(text.split())+epsilon))*100,3)
    pc_code_gloss = np.round(float(len([i for i in words if i in self.CodeGloss])/(len(text.split())+epsilon))*100,3)
    pc_att_markers = np.round(float(len([i for i in words if i in self.AttitudeMarkers])/(len(text.split())+epsilon))*100,3)
    pc_endophorics = np.round(float(len([i for i in words if i in self.Endophorics])/(len(text.split())+epsilon))*100,3)
    pc_hedges = np.round(float(len([i for i in words if i in self.Hedges])/(len(text.split())+epsilon))*100,3)
    pc_emphatics = np.round(float(len([i for i in words if i in self.Emphatics])/(len(text.split())+epsilon))*100,3)
    pc_frame_markers = np.round(float(len([i for i in words if i in self.FrameMarkers])/(len(text.split())+epsilon))*100,3)
    pc_evidentials = np.round(float(len([i for i in words if i in self.Evidentials])/(len(text.split())+epsilon))*100,3)

    pc_interactional = np.array([pc_person_markers, pc_hedges, pc_emphatics, pc_att_markers])
    pc_interactive = np.array([pc_code_gloss, pc_endophorics, pc_evidentials, pc_frame_markers])

    interactional = np.sum(pc_interactional, axis=0)
    interactive = np.sum(pc_interactive, axis=0)

    return np.round(interactional,3), np.round(interactive,3), len(words)

  def persuasiveness(self, text):
    epsilon = 0.000000000001;
    words = strip_multiple_whitespaces(strip_punctuation(text.lower())).split()
    pc_person_markers = np.round(float(len([i for i in words if i in self.PersonMarkers])/(len(text.split())+epsilon))*100,3)
    pc_code_gloss = np.round(float(len([i for i in words if i in self.CodeGloss])/(len(text.split())+epsilon))*100,3)
    pc_att_markers = np.round(float(len([i for i in words if i in self.AttitudeMarkers])/(len(text.split())+epsilon))*100,3)
    pc_endophorics = np.round(float(len([i for i in words if i in self.Endophorics])/(len(text.split())+epsilon))*100,3)
    pc_hedges = np.round(float(len([i for i in words if i in self.Hedges])/(len(text.split())+epsilon))*100,3)
    pc_emphatics = np.round(float(len([i for i in words if i in self.Emphatics])/(len(text.split())+epsilon))*100,3)
    pc_frame_markers = np.round(float(len([i for i in words if i in self.FrameMarkers])/(len(text.split())+epsilon))*100,3)
    pc_evidentials = np.round(float(len([i for i in words if i in self.Evidentials])/(len(text.split())+epsilon))*100,3)

    pc_interactive = np.array([pc_code_gloss, pc_endophorics, pc_evidentials, pc_frame_markers])

    return pc_interactive - (pc_hedges + pc_emphatics + pc_att_markers)


  def moving_average(self, x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

  # Auxiliary functions for preprocessing.
  def log(self, text):
    return print(text), print(strip_multiple_whitespaces(strip_punctuation(text.lower())).split())

  def get_sentences(self, text):
    mystr = text.replace(' ', '')
    return mystr.split(".")

  def get_sentences_from_txt(self, filename):
    sentences = []
    with open(filename, 'r') as f:
      for line in f:
        sentences.extend(line.split('.'))
    return sentences

## Basic analysis.

In [ ]:
# initialize a constructor
md = MetaDiscourseAnalysis()

In [ ]:
df = pd.read_csv("2023rice.csv")
df.head()


In [ ]:
df.columns

In [ ]:
df["pc_interactive"] = df["transcript"].apply(lambda x: md.authorial_stance(x)[1])
df["pc_interactional"] = df["transcript"].apply(lambda x: md.authorial_stance(x)[0])
df["persuasiveness"] = df["transcript"].apply(lambda x: md.persuasiveness(x)[0])
df["index"] = np.arange(len(df))

In [ ]:
# sort by persuasiveness ascending order

df.sort_values(by=['persuasiveness'], inplace=True, ascending=True)
df.head(4)

In [ ]:
px.scatter(df, x = "index", y="persuasiveness", hover_data="transcript")